# Predicting IQ

## Working with band ratios

### Getting dataframes with band ratios

In [1]:
# libraries
import numpy as np
import pandas as pd
import matplotlib as mpl 
import utils

In [2]:
def split_eyes_open_closed(feature_list, ret_non_matched = False): 

# input: feature list
# output: eyes-open features, eyes-closed features, non-matched features
    
    
    eyes_open, eyes_closed, non_matched = [], [], []
    for feature in feature_list: 
        if ('eyesopen' in feature) and ('eyesclosed' not in feature): 
            eyes_open.append(feature)
        elif ('eyesopen' not in feature) and ('eyesclosed' in feature): 
            eyes_closed.append(feature)
        else: 
            non_matched.append(feature)
    if ret_non_matched == True: 
        return eyes_open, eyes_closed, non_matched
    elif ret_non_matched == False:
        return eyes_open, eyes_closed
    else: 
        raise ValueError('ret_non_matched must be boolean')


In [3]:

def split_EEG_features_by_band (dataframe, band=None, to_escape_features = ['id','Age', 'DX_01_Cat', 'DX_01_Sub', 'DX_01','quality_rating']):
    
# input: pandas dataframe , band = ['alpha', 'beta','gamma', 'delta', 'theta', 'mu'], features to escape
# output:  1) dataframe with features that are ONLY relative to the selected band, i.e. MIXED FEATURES ARE DISCARDED
#          2) list of features relative to the selected band (escaped features excluded)
    to_remove_features = []
    bands = ['alpha', 'beta','gamma', 'delta', 'theta', 'mu']
    if band == None: 
        raise ValueError('you must select a valid band: alpha, beta, gamma, delta, theta, mu')
    elif band not in bands: 
        raise ValueError('you must select a valid band: alpha, beta, gamma, delta, theta, mu')
    else:
        not_selected_bands = [item for item in bands if item != band]
        for feature in list(dataframe.columns):
            if feature in to_escape_features:
                continue
            else:
                if band not in feature: 
                    to_remove_features.append(feature)
        new_dataframe = dataframe.drop(to_remove_features, axis=1)
        band_features =[e for e in list(new_dataframe.columns) if e not in to_escape_features]
    return new_dataframe, band_features

In [4]:
eeg_data = utils.create_dataset_eeg(clusters=True)    # eeg_data with clusters from utils function

eeg_clusters = pd.read_csv("data/EEG/resting_eeg_clusters.csv") # eeg clusters data 

/Users/Proprietario/opt/anaconda3/envs/dslab1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (50,78,80,91,92,93,94,95,105,106,107,108,109,119,120,121,123,133,134,135,137,276,291,292,295,296,297,300,301,302,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
bands = ['alpha', 'beta','gamma', 'delta', 'theta', 'mu'] #list of bands for EEG

basic_features_eeg_data = [ 'id','Age', 'DX_01_Cat', 'DX_01_Sub', 'DX_01','quality_rating']  # basic features in eeg data

eeg_clusters_features = list(eeg_clusters.columns)
eeg_clusters_features.remove('id')   #removing features in common with basic features
eeg_clusters_features.remove('quality_rating') #removing features in common with basic features

eeg_data_clusters = eeg_data[basic_features_eeg_data + eeg_clusters_features]

In [6]:
clu_features_dict = {}
for band in bands:
    clu_features_dict[band]={
        'eyesopen' : split_eyes_open_closed(split_EEG_features_by_band(eeg_data_clusters, band)[1])[0],
        'eyesclosed' : split_eyes_open_closed(split_EEG_features_by_band(eeg_data_clusters, band)[1])[0]
    }

In [7]:
    
def get_band_ratio_dataframe(kind,eyes): 
    
    if(eyes != 'eyesclosed' and eyes != 'eyesopen'):
        raise ValueError('incorrect value for "eyes"')
    
    if kind == 'betatheta': 
        band1 = 'beta'
        band2 = 'theta'
    elif kind == 'thetabeta':
        band1 = 'theta'
        band2 = 'beta'
    elif kind == 'betaalpha': 
        band1 = 'beta'
        band2 = 'alpha'
    elif kind == 'alphabeta':
        band1 = 'alpha'
        band2 = 'beta'
    elif kind == 'thetaalpha': 
        band1 = 'theta'
        band2 = 'alpha'
    elif kind == 'alphatheta':
        band1 = 'alpha'
        band2 = 'theta'
    elif kind == 'deltagamma': 
        band1 = 'delta'
        band2 = 'gamma'
    elif kind == 'gammadelta':
        band1 = 'gamma'
        band2 = 'delta'
    else: 
        raise ValueError("incorrect value for 'kind'")
        

    b1_b2_ratio_df = eeg_data_clusters[basic_features_eeg_data]
    
    if band1 == 'alpha': 
        eyes_b1_b2 = tuple(zip(clu_features_dict[band1][eyes][:48], clu_features_dict[band2][eyes]))
    elif band2 == 'alpha': 
        eyes_b1_b2 = tuple(zip(clu_features_dict[band1][eyes], clu_features_dict[band2][eyes][:48]))
    else: 
        eyes_b1_b2 = tuple(zip(clu_features_dict[band1][eyes], clu_features_dict[band2][eyes]))
        

    for couple in eyes_b1_b2:


        b1_column = pd.DataFrame(eeg_data_clusters[couple[0]]).astype(dtype='float64')
        b2_column = pd.DataFrame(eeg_data_clusters[couple[1]]).astype(dtype='float64')

        b1_b2_ratio_column = np.divide(b1_column.values,b2_column.values)

        b1_b2_ratio_column= pd.DataFrame(b1_b2_ratio_column, index=b1_column.index,columns=[couple[0].replace(band1, band1+band2)])
        b1_b2_ratio_df = pd.concat([b1_b2_ratio_df, b1_b2_ratio_column], axis=1)

    return b1_b2_ratio_df

In [10]:
get_band_ratio_dataframe('alphabeta','eyesopen')

,id,Age,DX_01_Cat,DX_01_Sub,DX_01,quality_rating,eyesopen_fband_alphabeta_absmean_lfront,eyesopen_fband_alphabeta_absmean_mfront,eyesopen_fband_alphabeta_absmean_rfront,eyesopen_fband_alphabeta_absmean_lpari,...,eyesopen_alphabetapeak_derivative_freq,eyesopen_alphabetapeak_derivative_amplitude,eyesopen_alphabetapeak_gravity_freq,eyesopen_alphabetapeak_gravity_amplitude,eyesopen_indfband_lower1alphabeta_absmean_lfront,eyesopen_indfband_lower1alphabeta_absmean_mfront,eyesopen_indfband_lower1alphabeta_absmean_rfront,eyesopen_indfband_lower1alphabeta_absmean_lpari,eyesopen_indfband_lower1alphabeta_absmean_mpari,eyesopen_indfband_lower1alphabeta_absmean_rpari
0,NDARFM080VAF,7.623203,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,O,7.086405,9.056246,9.075677,13.117117,...,27.878677,17.942878,25.102979,12.764890,5.187392,4.936250,4.652087,8.689205,10.922404,9.064319
1,NDARLE953JYU,12.357403,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Inattentive Type,G,3.925508,1.953529,2.297587,2.924569,...,17.953788,2.367046,13.595892,3.544596,1.444683,1.584761,0.879338,1.082826,1.207756,0.842134
2,NDARCP360AFD,8.357061,Depressive Disorders,NaN,Disruptive Mood Dysregulation Disorder,G,3.726002,4.637523,4.101270,4.927524,...,50.506628,5.614768,36.295673,3.565819,2.868122,4.191720,3.450911,3.862896,7.174369,4.165216
3,NDARJT730WP0,14.996235,Anxiety Disorders,NaN,Panic Disorder,B,3.631657,5.413444,2.990026,2.600447,...,390.754170,4.864867,71.552042,1.995244,0.090705,0.067482,0.081529,0.251115,0.374778,0.288671
4,NDARHZ439EBY,7.218571,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,O,12.501618,14.145445,13.826050,21.552882,...,35.464283,34.078100,17.022087,34.881470,7.748553,12.867109,6.989182,17.450349,29.116031,7.236897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,NDARKK597VDH,13.847022,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,O,10.117757,11.470612,15.471410,14.591141,...,51.270403,20.780425,38.750123,13.521333,2.924294,4.505178,3.994123,2.440903,4.485248,3.375165
783,NDARDE827UF0,17.101756,Depressive Disorders,NaN,Major Depressive Disorder,B,2.619821,2.451711,2.491158,2.421460,...,37.604633,3.086996,29.209716,4.052575,0.586802,0.729316,1.417743,0.989473,1.540432,0.394729
784,NDARND697FLK,6.517796,No Diagnosis Given,NaN,No Diagnosis Given,B,3.583192,3.736034,3.270500,3.058776,...,59.095490,1.415751,41.817162,6.334061,6.900302,12.172353,4.955173,7.076709,8.195374,3.767052
785,NDARCL008HLA,16.456194,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,B,28.682729,28.118563,35.837836,39.242996,...,29.492020,47.600168,25.444711,26.442804,5.702371,9.706685,8.153705,7.471653,10.096514,8.075536
